In [64]:
import os
import requests
import pandas as pd
from sqlalchemy import create_engine, text, types
from dotenv import load_dotenv

# Configuração para exibir todas as colubas do DataFrame
pd.set_option("display.max_columns", None)

In [65]:
# Carregar as variáveis de ambiente (credenciais do DB) do arquivo .env
load_dotenv()

# Constantes do projeto
API_URL_MUNICIPIOS = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
RAW_DATA_PATH = "dados_brutos_municipios.csv"
ENRICHMENT_DATA_PATH = "regioes_enriquecimento.csv"
DB_TABLE_NAME = "municipios_brasil"

print("Variáveis de ambiente carregadas com sucesso")


Variáveis de ambiente carregadas com sucesso


## Extração

In [66]:
print("Iniciando a extração dos dados da API do IBGE")

try:
    response = requests.get(API_URL_MUNICIPIOS, timeout=30)
    response.raise_for_status() # garante que a requisição foi bem-sucedida

    data = response.json()
    df_raw = pd.DataFrame(data)

    # Salvar uma cópia bruta para backup e auditoria
    df_raw.to_csv(RAW_DATA_PATH, index=False, encoding='utf-8')

    print(f"Extração concluída! {len(df_raw)} registros extraídos")
    print(f"Backup dos dados brutos salvo em '{RAW_DATA_PATH}")

except requests.exceptions.RequestException as e:
    print(f"Erro na extração: {e}")
    df_raw = pd.DataFrame() # cria um dataframe vazio em caso de erro

Iniciando a extração dos dados da API do IBGE
Extração concluída! 5571 registros extraídos
Backup dos dados brutos salvo em 'dados_brutos_municipios.csv


## Exploração e Limpeza dos dados

In [67]:
# Infomações gerais dos dados
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5571 entries, 0 to 5570
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               5571 non-null   int64 
 1   nome             5571 non-null   object
 2   microrregiao     5570 non-null   object
 3   regiao-imediata  5571 non-null   object
dtypes: int64(1), object(3)
memory usage: 174.2+ KB


In [68]:
# Verificar valores nulos
print(df_raw.isnull().sum())

print("\n")
df_raw.head()

id                 0
nome               0
microrregiao       1
regiao-imediata    0
dtype: int64




,id,nome,microrregiao,regiao-imediata
0,1100015,Alta Floresta D'Oeste,"{'id': 11006, 'nome': 'Cacoal', 'mesorregiao':...","{'id': 110005, 'nome': 'Cacoal', 'regiao-inter..."
1,1100023,Ariquemes,"{'id': 11003, 'nome': 'Ariquemes', 'mesorregia...","{'id': 110002, 'nome': 'Ariquemes', 'regiao-in..."
2,1100031,Cabixi,"{'id': 11008, 'nome': 'Colorado do Oeste', 'me...","{'id': 110006, 'nome': 'Vilhena', 'regiao-inte..."
3,1100049,Cacoal,"{'id': 11006, 'nome': 'Cacoal', 'mesorregiao':...","{'id': 110005, 'nome': 'Cacoal', 'regiao-inter..."
4,1100056,Cerejeiras,"{'id': 11008, 'nome': 'Colorado do Oeste', 'me...","{'id': 110006, 'nome': 'Vilhena', 'regiao-inte..."


In [69]:
# Criar uma cópia para preservar o DataFrame original
df_cleaned = df_raw.copy()

# Renomear colunas
df_cleaned = df_cleaned.rename(columns={'id': 'id_municipio', 'nome': 'nome_municipio'})

df_cleaned.head()

,id_municipio,nome_municipio,microrregiao,regiao-imediata
0,1100015,Alta Floresta D'Oeste,"{'id': 11006, 'nome': 'Cacoal', 'mesorregiao':...","{'id': 110005, 'nome': 'Cacoal', 'regiao-inter..."
1,1100023,Ariquemes,"{'id': 11003, 'nome': 'Ariquemes', 'mesorregia...","{'id': 110002, 'nome': 'Ariquemes', 'regiao-in..."
2,1100031,Cabixi,"{'id': 11008, 'nome': 'Colorado do Oeste', 'me...","{'id': 110006, 'nome': 'Vilhena', 'regiao-inte..."
3,1100049,Cacoal,"{'id': 11006, 'nome': 'Cacoal', 'mesorregiao':...","{'id': 110005, 'nome': 'Cacoal', 'regiao-inter..."
4,1100056,Cerejeiras,"{'id': 11008, 'nome': 'Colorado do Oeste', 'me...","{'id': 110006, 'nome': 'Vilhena', 'regiao-inte..."


In [70]:
# Verificar e remover duplicatas
duplicatas_antes = df_cleaned.duplicated(subset=['id_municipio']).sum()
print(f" Verificando duplicatas... existe {duplicatas_antes} registro(s) duplicado(s)\n")

if duplicatas_antes > 0:
    df_cleaned = df_cleaned.drop_duplicates(subset=['id_municipio'], keep='first')
    print("Duplicatas removidas\n")

# Visualizando os dados
display(df_cleaned)

 Verificando duplicatas... existe 0 registro(s) duplicado(s)



,id_municipio,nome_municipio,microrregiao,regiao-imediata
0,1100015,Alta Floresta D'Oeste,"{'id': 11006, 'nome': 'Cacoal', 'mesorregiao':...","{'id': 110005, 'nome': 'Cacoal', 'regiao-inter..."
1,1100023,Ariquemes,"{'id': 11003, 'nome': 'Ariquemes', 'mesorregia...","{'id': 110002, 'nome': 'Ariquemes', 'regiao-in..."
2,1100031,Cabixi,"{'id': 11008, 'nome': 'Colorado do Oeste', 'me...","{'id': 110006, 'nome': 'Vilhena', 'regiao-inte..."
3,1100049,Cacoal,"{'id': 11006, 'nome': 'Cacoal', 'mesorregiao':...","{'id': 110005, 'nome': 'Cacoal', 'regiao-inter..."
4,1100056,Cerejeiras,"{'id': 11008, 'nome': 'Colorado do Oeste', 'me...","{'id': 110006, 'nome': 'Vilhena', 'regiao-inte..."
...,...,...,...,...
5566,5222005,Vianópolis,"{'id': 52016, 'nome': 'Pires do Rio', 'mesorre...","{'id': 520002, 'nome': 'Anápolis', 'regiao-int..."
5567,5222054,Vicentinópolis,"{'id': 52015, 'nome': 'Meia Ponte', 'mesorregi...","{'id': 520009, 'nome': 'Piracanjuba', 'regiao-..."
5568,5222203,Vila Boa,"{'id': 52012, 'nome': 'Entorno de Brasília', '...","{'id': 520022, 'nome': 'Flores de Goiás', 'reg..."
5569,5222302,Vila Propício,"{'id': 52012, 'nome': 'Entorno de Brasília', '...","{'id': 520018, 'nome': 'Ceres - Rialma - Goian..."


In [71]:
#df_normalized = pd.json_normalize(df_cleaned['microrregiao'])
#df_normalized.head(5)

## Transformação dos dados

In [72]:
# Achatar a coluna 'microrregiao' que contém um dicionário
try:
    df_normalized = pd.json_normalize(df_cleaned['microrregiao'])

    # criar novas colunas a partir dos dados extraídos
    df_cleaned['id_uf'] = df_normalized['mesorregiao.UF.id']
    df_cleaned['sigla_uf'] = df_normalized['mesorregiao.UF.sigla']
    df_cleaned['nome_uf'] = df_normalized['mesorregiao.UF.nome']
    

    # Remover coluna original que já não é necessária
    df_cleaned = df_cleaned.drop(columns='microrregiao')

    # Corrigir os tipos de dados
    df_cleaned['id_municipio'] = pd.to_numeric(df_cleaned['id_municipio'], errors='coerce')
    df_cleaned['id_uf'] = pd.to_numeric(df_cleaned['id_uf'], errors='coerce')

    print("\n---- Visualizando os dados transformados: ----")
    display(df_cleaned.head())

    print("\n---- Verificando os tipos de dados ----")
    df_cleaned.info()

except Exception as e:
    print(f"Erro na transformação: {e}")


---- Visualizando os dados transformados: ----


,id_municipio,nome_municipio,regiao-imediata,id_uf,sigla_uf,nome_uf
0,1100015,Alta Floresta D'Oeste,"{'id': 110005, 'nome': 'Cacoal', 'regiao-inter...",11.0,RO,Rondônia
1,1100023,Ariquemes,"{'id': 110002, 'nome': 'Ariquemes', 'regiao-in...",11.0,RO,Rondônia
2,1100031,Cabixi,"{'id': 110006, 'nome': 'Vilhena', 'regiao-inte...",11.0,RO,Rondônia
3,1100049,Cacoal,"{'id': 110005, 'nome': 'Cacoal', 'regiao-inter...",11.0,RO,Rondônia
4,1100056,Cerejeiras,"{'id': 110006, 'nome': 'Vilhena', 'regiao-inte...",11.0,RO,Rondônia



---- Verificando os tipos de dados ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5571 entries, 0 to 5570
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_municipio     5571 non-null   int64  
 1   nome_municipio   5571 non-null   object 
 2   regiao-imediata  5571 non-null   object 
 3   id_uf            5570 non-null   float64
 4   sigla_uf         5570 non-null   object 
 5   nome_uf          5570 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 261.3+ KB
